# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,17.60,86,34,2.01,US,1693135407
1,1,invercargill,-46.4000,168.3500,1.68,91,69,1.34,NZ,1693135407
2,2,dudinka,69.4058,86.1778,15.90,63,51,5.05,RU,1693135408
3,3,ust-nera,64.5667,143.2000,10.58,77,98,2.50,RU,1693135409
4,4,noshiro,40.2039,140.0242,25.68,85,100,1.77,JP,1693135409


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)
# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"] >= 25) & 
                            (city_data_df["Max Temp"] < 35).dropna()]

# Display sample data
display(ideal_city)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,noshiro,40.2039,140.0242,25.68,85,100,1.77,JP,1693135409
8,8,atafu village,-8.5421,-172.5159,28.20,72,82,5.77,TK,1693135415
9,9,luderitz,-26.6481,15.1594,26.16,20,0,5.85,NaN,1693135416
16,16,abadeh,31.1608,52.6506,31.30,16,0,4.65,IR,1693135423
21,21,touros,-5.1989,-35.4608,27.80,71,4,4.14,BR,1693135426
...,...,...,...,...,...,...,...,...,...,...
537,537,ust'-charyshskaya pristan',52.3934,83.6635,25.86,42,85,2.62,RU,1693135764
540,540,souillac,-20.5167,57.5167,26.21,65,40,6.17,MU,1693135766
542,542,tartouss,34.8890,35.8866,29.56,65,20,7.53,SY,1693135768
548,548,kili,5.6439,169.1211,28.71,72,66,3.09,MH,1693135772


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,noshiro,JP,40.2039,140.0242,85,
8,atafu village,TK,-8.5421,-172.5159,72,
9,luderitz,NaN,-26.6481,15.1594,20,
16,abadeh,IR,31.1608,52.6506,16,
21,touros,BR,-5.1989,-35.4608,71,
...,...,...,...,...,...,...
537,ust'-charyshskaya pristan',RU,52.3934,83.6635,42,
540,souillac,MU,-20.5167,57.5167,65,
542,tartouss,SY,34.8890,35.8866,65,
548,kili,MH,5.6439,169.1211,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {"categories": categories,
"apiKey": geoapify_key}
    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    hotel_api = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    hotel_api = hotel_api.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_api["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
noshiro - nearest hotel: あづま旅館
atafu village - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
abadeh - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
cockburn town - nearest hotel: The Salt Raker Inn
sokhumi - nearest hotel: Монэ
port isabel - nearest hotel: Queen Isabel Inn
nola - nearest hotel: Pamaran
hobyo - nearest hotel: No hotel found
ihosy - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
nandgaon - nearest hotel: No hotel found
kintampo - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
adra - nearest hotel: No hotel found
archidona - nearest hotel: Hotel Escuela Convento Santo Domingo
anloga - nearest hotel: Hotel de White House
rabaul - nearest hotel: Rabaul Hotel
tolanaro - nearest hotel: Hôtel Mahavokey
mili - nearest hotel: No hotel found
puerto penasco - nearest hotel: Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
4,noshiro,JP,40.2039,140.0242,85,あづま旅館
8,atafu village,TK,-8.5421,-172.5159,72,No hotel found
9,luderitz,NaN,-26.6481,15.1594,20,Seaview Hotel Zum Sperrgebiet
16,abadeh,IR,31.1608,52.6506,16,No hotel found
21,touros,BR,-5.1989,-35.4608,71,Pousada Atlântico
...,...,...,...,...,...,...
537,ust'-charyshskaya pristan',RU,52.3934,83.6635,42,Гостиница
540,souillac,MU,-20.5167,57.5167,65,Hotel SkylineInn
542,tartouss,SY,34.8890,35.8866,65,فندق انترادوس
548,kili,MH,5.6439,169.1211,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)